# EV Sweep (Severity) and Two EV Comparisons

In this notebook I do two things:

Part A: Sweep EV size at a fixed location (usually the weakest bus)
- EV sizes: 3.6, 7, 11, 22, 30 kW (can be changed)
- Record minimum voltage and violation counts

Part B: Compare two EV placements (same EV size)
- Case 1: both EVs at weakest bus (clustered)
- Case 2: one EV at weakest bus and one at source bus (distributed)

Outputs:
- Topology plot for fixed location
- Sweep plots: min V vs size, violations vs size
- Topology plots for two EV cases
- Voltage profiles for two EV cases


In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using PowerModelsDistribution
using DataFrames
using Ipopt
using JuMP
using LinearAlgebra
using Plots
using Graphs
using CairoMakie
using Statistics

const PMD = PowerModelsDistribution

file = "../data/networks/lv_small_feeder/Master.dss"
fig_dir = "../results/figures"
mkpath(fig_dir)


  Activating project at `/mnt/c/Users/auc009/OneDrive - CSIRO/Documents/power-models-distribution/pmd_ev_experiments`


"../results/figures"

## Reuse helper functions

Copy the helper function block from 02_single_ev.ipynb into this notebook.


In [2]:
function build_lines_df(eng)
    line_dict = get(eng, "line", Dict())
    rows = NamedTuple[]
    for (id, ln) in line_dict
        bus1 = get(ln, "bus1", get(ln, "f_bus", nothing))
        bus2 = get(ln, "bus2", get(ln, "t_bus", nothing))
        if bus1 === nothing || bus2 === nothing
            continue
        end
        b1 = split(String(bus1), ".")[1]
        b2 = split(String(bus2), ".")[1]
        L = float(get(ln, "length", 1.0))
        push!(rows, (Bus1=b1, Bus2=b2, Length=L))
    end
    return DataFrame(rows)
end

function build_graph_and_weights(lines_df::DataFrame)
    buses = unique(vcat(lines_df.Bus1, lines_df.Bus2))
    sort!(buses)
    b2i = Dict(b => i for (i,b) in enumerate(buses))
    i2b = Dict(i => b for (b,i) in b2i)

    g = Graph(length(buses))
    w = Dict{Tuple{Int,Int}, Float64}()

    for r in eachrow(lines_df)
        i = b2i[r.Bus1]
        j = b2i[r.Bus2]
        add_edge!(g, i, j)
        w[(i,j)] = r.Length
        w[(j,i)] = r.Length
    end
    return g, w, b2i, i2b
end

function dijkstra_distances(g::Graph, w::Dict{Tuple{Int,Int},Float64}, s::Int)
    n = nv(g)
    dist = fill(Inf, n)
    prev = fill(0, n)
    visited = falses(n)

    dist[s] = 0.0
    pq = [(0.0, s)]

    while !isempty(pq)
        sort!(pq, by=x->x[1])
        (d,u) = popfirst!(pq)
        if visited[u]
            continue
        end
        visited[u] = true

        for v in neighbors(g, u)
            nd = d + get(w, (u,v), 1.0)
            if nd < dist[v]
                dist[v] = nd
                prev[v] = u
                push!(pq, (nd, v))
            end
        end
    end
    return dist, prev
end

function tree_children(prev::Vector{Int})
    n = length(prev)
    children = [Int[] for _ in 1:n]
    for v in 1:n
        p = prev[v]
        if p != 0
            push!(children[p], v)
        end
    end
    return children
end

function tree_layout(prev::Vector{Int}, dist_km::Vector{Float64}, root::Int)
    children = tree_children(prev)
    y = fill(0.0, length(prev))
    next_y = Ref(0.0)

    function dfs(u::Int)
        if isempty(children[u])
            y[u] = next_y[]
            next_y[] += 1.0
        else
            for c in sort(children[u])
                dfs(c)
            end
            y[u] = mean(y[children[u]])
        end
    end

    dfs(root)
    x = copy(dist_km)
    return x, y
end

function plot_topology_with_markers(
    g::Graph, xpos::Vector{Float64}, ypos::Vector{Float64}, dist_km::Vector{Float64};
    source_idx::Int,
    ev_indices::Vector{Int}=Int[],
    title_str::String="Topology (colored by distance)",
    save_path::Union{Nothing,String}=nothing
)
    fig = Figure(size=(1000, 700))
    ax = Axis(fig[1,1], title=title_str)

    for e in edges(g)
        u = src(e); v = dst(e)
        lines!(ax, [xpos[u], xpos[v]], [ypos[u], ypos[v]], color=:gray50, linewidth=2)
    end

    scatter!(ax, xpos, ypos, color=dist_km, markersize=10)
    scatter!(ax, [xpos[source_idx]], [ypos[source_idx]], markersize=22, color=:black)
    text!(ax, "source", position=(xpos[source_idx], ypos[source_idx] + 0.6), align=(:center, :bottom))

    for (k, ev) in enumerate(ev_indices)
        scatter!(ax, [xpos[ev]], [ypos[ev]], markersize=22, color=:red)
        text!(ax, "EV$(k)", position=(xpos[ev], ypos[ev] + 0.6), align=(:center, :bottom))
    end

    hidedecorations!(ax)
    hidespines!(ax)

    if save_path !== nothing
        save(save_path, fig)
    end
    return fig
end

function busid_to_name_map(math)
    m = Dict{String,String}()
    for (id, b) in math["bus"]
        m[string(id)] = split(get(b, "name", string(id)), ".")[1]
    end
    return m
end

function extract_vm_by_busid(result)
    vm = Dict{String, Vector{Float64}}()
    for (id, b) in result["solution"]["bus"]
        vr = b["vr"]; vi = b["vi"]
        vm[string(id)] = sqrt.(vr.^2 .+ vi.^2)
    end
    return vm
end

function build_buses_dict_for_voltage_plot(id2name::Dict{String,String}, dist_km_byname::Dict{String,Float64}, vm_byid::Dict{String,Vector{Float64}})
    buses_dict = Dict{String, Dict{String,Any}}()
    for (id, vm) in vm_byid
        name = id2name[id]
        vma = length(vm) >= 1 ? vm[1] : NaN
        vmb = length(vm) >= 2 ? vm[2] : NaN
        vmc = length(vm) >= 3 ? vm[3] : NaN
        buses_dict[name] = Dict(
            "distance" => get(dist_km_byname, name, NaN),
            "vma" => [float(vma)],
            "vmb" => [float(vmb)],
            "vmc" => [float(vmc)]
        )
    end
    return buses_dict
end

function plot_voltage_along_feeder_snap(buses_dict, lines_df; t=1, vmin=0.9, vmax=1.1)
    p = plot(legend=false)
    xlabel!("Electrical distance from source (km)")
    ylabel!("Voltage magnitude (p.u.)")
    title!("Voltage drop along feeder (snapshot)")

    colors = [:blue, :red, :black]

    for r in eachrow(lines_df)
        b1 = r.Bus1
        b2 = r.Bus2
        for phase in 1:3
            d1 = buses_dict[b1]["distance"]
            d2 = buses_dict[b2]["distance"]
            v1 = phase==1 ? buses_dict[b1]["vma"][t] : phase==2 ? buses_dict[b1]["vmb"][t] : buses_dict[b1]["vmc"][t]
            v2 = phase==1 ? buses_dict[b2]["vma"][t] : phase==2 ? buses_dict[b2]["vmb"][t] : buses_dict[b2]["vmc"][t]
            if isfinite(d1) && isfinite(d2) && isfinite(v1) && isfinite(v2)
                plot!([d1, d2], [v1, v2], color=colors[phase], marker=:circle, markersize=1)
            end
        end
    end

    maxdist = maximum([b["distance"] for (k,b) in buses_dict if isfinite(b["distance"])])
    plot!([0, maxdist], [vmin, vmin], color=:red, linestyle=:dash)
    plot!([0, maxdist], [vmax, vmax], color=:red, linestyle=:dash)
    return p
end

function add_ev_load!(math, bus_id::String; phase::Int=1, P_kw::Float64=7.0, Q_kvar::Float64=0.0)
    sbase = get(math["settings"], "sbase", 1.0)
    P_pu = P_kw / (sbase * 1000)
    Q_pu = Q_kvar / (sbase * 1000)

    existing = parse.(Int, collect(keys(math["load"])))
    new_id = string(isempty(existing) ? 1 : maximum(existing) + 1)

    math["load"][new_id] = Dict(
        "bus" => bus_id,
        "connections" => [phase],
        "pd" => [P_pu],
        "qd" => [Q_pu],
        "status" => 1,
        "model" => "constant_power"
    )
    return new_id
end

function find_math_bus_id_by_name(id2name::Dict{String,String}, target_name::String)
    for (id, nm) in id2name
        if nm == target_name
            return id
        end
    end
    error("No math bus id found for name '$target_name'")
end

find_math_bus_id_by_name (generic function with 1 method)

In [ ]:
eng4w = PMD.parse_file(file, transformations=[PMD.transform_loops!])
eng4w["settings"]["sbase_default"] = 1.0 * 1e3 / eng4w["settings"]["power_scale_factor"]
PMD.add_bus_absolute_vbounds!(eng4w, phase_lb_pu=0.9, phase_ub_pu=1.1, neutral_ub_pu=0.1)

lines_df = build_lines_df(eng4w)
g, w, b2i, i2b = build_graph_and_weights(lines_df)

source_name = "sourcebus"
source_idx = b2i[source_name]
dist_raw, prev = dijkstra_distances(g, w, source_idx)

scale_to_km = 1.0 / 1000
dist_km = dist_raw .* scale_to_km
xpos, ypos = tree_layout(prev, dist_km, source_idx)
dist_km_byname = Dict(i2b[i] => dist_km[i] for i in 1:length(dist_km) if isfinite(dist_km[i]))

math_base = PMD.transform_data_model(eng4w; multinetwork=false, kron_reduce=false, phase_project=false)
id2name = busid_to_name_map(math_base)

res_base = PMD.solve_mc_opf(math_base, IVRENPowerModel, Ipopt.Optimizer)
vm_base = extract_vm_by_busid(res_base)
buses_dict_base = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vm_base)

p_base = plot_voltage_along_feeder_snap(buses_dict_base, lines_df; vmin=0.9, vmax=1.1)
savefig(p_base, joinpath(fig_dir, "04_voltage_profile_baseline.png"))

source_bus_id = string(math_base["bus_lookup"]["sourcebus"])
source_bus_name = id2name[source_bus_id]

# Weakest bus
minV_base_byid = Dict(id => minimum(vm[1:min(3, length(vm))]) for (id, vm) in vm_base)
weakest_bus_id = first(keys(minV_base_byid))
weakest_val = minV_base_byid[weakest_bus_id]
for (id, v) in minV_base_byid
    if v < weakest_val
        weakest_val = v
        weakest_bus_id = id
    end
end
weakest_name = id2name[weakest_bus_id]

println("Weakest bus: ", weakest_name, " minV=", weakest_val)

## Part A: EV severity sweep at fixed location

I fix the EV at the weakest bus and increase EV power.


In [ ]:
fixed_bus_name = weakest_name
fixed_bus_id = weakest_bus_id
fixed_bus_idx = b2i[fixed_bus_name]

plot_topology_with_markers(
    g, xpos, ypos, dist_km;
    source_idx=source_idx,
    ev_indices=[fixed_bus_idx],
    title_str="Topology with fixed EV location (sweep at weakest bus)",
    save_path=joinpath(fig_dir, "04_topology_sweep_fixed_weakest.png")
)

In [ ]:
P_levels_kw = [3.6, 7.0, 11.0, 22.0, 30.0]
ev_phase = 1

rows = NamedTuple[]

for PkW in P_levels_kw
    math = deepcopy(math_base)
    add_ev_load!(math, fixed_bus_id; phase=ev_phase, P_kw=PkW, Q_kvar=0.0)

    res = PMD.solve_mc_opf(math, IVRENPowerModel, Ipopt.Optimizer)
    vm = extract_vm_by_busid(res)

    push!(rows, (
        P_kw=PkW,
        status=string(res["termination_status"]),
        minV=min_phase_voltage(vm),
        violations=count_voltage_violations(vm; vmin=0.9, vmax=1.1)
    ))
end

sweep_df = DataFrame(rows)
sweep_df

In [ ]:
p1 = plot(sweep_df.P_kw, sweep_df.minV,
    xlabel="EV power (kW)",
    ylabel="Minimum phase voltage (p.u.)",
    title="EV severity sweep at weakest bus",
    legend=false,
    marker=:circle
)
hline!([0.9], linestyle=:dash)
savefig(p1, joinpath(fig_dir, "04_sweep_minV_vs_P.png"))

p2 = plot(sweep_df.P_kw, sweep_df.violations,
    xlabel="EV power (kW)",
    ylabel="Number of buses with voltage violation",
    title="Voltage violations vs EV power",
    legend=false,
    marker=:circle
)
savefig(p2, joinpath(fig_dir, "04_sweep_violations_vs_P.png"))

## Part B: Two EV comparisons

I compare clustered vs distributed EV placement.

Case A: two EVs at weakest bus
Case B: one EV at weakest bus and one EV at source bus

I use the same EV size for each EV (for example 7 kW).


In [ ]:
P_two_kw = 7.0
ev_phase = 1

# Case A: clustered
caseA = deepcopy(math_base)
add_ev_load!(caseA, weakest_bus_id; phase=ev_phase, P_kw=P_two_kw, Q_kvar=0.0)
add_ev_load!(caseA, weakest_bus_id; phase=ev_phase, P_kw=P_two_kw, Q_kvar=0.0)

resA = PMD.solve_mc_opf(caseA, IVRENPowerModel, Ipopt.Optimizer)
vmA = extract_vm_by_busid(resA)
busesA = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vmA)

# Case B: distributed
caseB = deepcopy(math_base)
add_ev_load!(caseB, weakest_bus_id; phase=ev_phase, P_kw=P_two_kw, Q_kvar=0.0)
add_ev_load!(caseB, source_bus_id;  phase=ev_phase, P_kw=P_two_kw, Q_kvar=0.0)

resB = PMD.solve_mc_opf(caseB, IVRENPowerModel, Ipopt.Optimizer)
vmB = extract_vm_by_busid(resB)
busesB = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vmB)

# Topology plots
weak_idx = b2i[weakest_name]
src_idx2 = b2i[source_bus_name]

plot_topology_with_markers(
    g, xpos, ypos, dist_km;
    source_idx=source_idx,
    ev_indices=[weak_idx, weak_idx],
    title_str="Two EVs clustered at weakest bus",
    save_path=joinpath(fig_dir, "04_topology_twoEV_clustered.png")
)

plot_topology_with_markers(
    g, xpos, ypos, dist_km;
    source_idx=source_idx,
    ev_indices=[weak_idx, src_idx2],
    title_str="Two EVs distributed (weakest + source)",
    save_path=joinpath(fig_dir, "04_topology_twoEV_distributed.png")
)

# Voltage profile plots
pA = plot_voltage_along_feeder_snap(busesA, lines_df; vmin=0.9, vmax=1.1)
plot!(pA, p_base)
savefig(pA, joinpath(fig_dir, "04_voltage_profile_twoEV_clustered.png"))

pB = plot_voltage_along_feeder_snap(busesB, lines_df; vmin=0.9, vmax=1.1)
plot!(pB, p_base)
savefig(pB, joinpath(fig_dir, "04_voltage_profile_twoEV_distributed.png"))

println("Case A minV: ", min_phase_voltage(vmA), " violations: ", count_voltage_violations(vmA))
println("Case B minV: ", min_phase_voltage(vmB), " violations: ", count_voltage_violations(vmB))

## Notes

This notebook tells me:
- how voltage worsens as EV size increases at a stressed location
- whether clustering EVs is worse than spreading them out

Next step after this is to pick a stressed case where voltage limits are violated.
That case becomes the input scenario for STATCOM experiments.
